# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Import dataset

## ML-100K Dataset

In [3]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-07-18 18:18:50--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.5MB/s    in 0.3s    

2020-07-18 18:18:50 (15.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

## ML-1M Dataset

In [4]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-07-18 18:18:58--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  18.1MB/s    in 0.3s    

2020-07-18 18:18:58 (18.1 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


# Data preprocessing

## Preparing the Training Set and Test Set

In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

## Get the Number of Users and Movies

In [6]:
num_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
num_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

## Convert data such that users are rows and movies are columns

In [7]:
def convert(data):
  new_data = []
  for id_user in range(1, num_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_user]
    id_ratings = data[:, 2][data[:, 0] == id_user]
    ratings = np.zeros(num_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

## Convert data to torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the architecture of the Neural Network 

In [12]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()    #inherit form the nn module
    # encoding
    self.fc1 = nn.Linear(num_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    # decoding
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, num_movies)
    #activation function
    self.activation = nn.Sigmoid()
  def forward(self, X):
    X = self.activation(self.fc1(X))
    X = self.activation(self.fc2(X))
    X = self.activation(self.fc3(X))
    X = self.fc4(X)
    return X


# Create an SAE Object

In [13]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Train the SAE model

In [15]:
num_epoch = 200
for epoch in range(1, num_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(num_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = num_movies/(float(torch.sum(target.data > 0) + 1e-10))
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7660)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0382)
epoch: 5 loss: tensor(1.0310)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0218)
epoch: 9 loss: tensor(1.0208)
epoch: 10 loss: tensor(1.0197)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0184)
epoch: 13 loss: tensor(1.0178)
epoch: 14 loss: tensor(1.0174)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0166)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0162)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0159)
epoch: 26 loss: tensor(1.0155)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0139)
epoch: 30 loss: tensor(1.0120)
epoch: 31 loss: tensor(1.0097)
epoch: 32 loss: tensor(1.0088)
epoch: 33 loss: t

## Testing the SAE model

In [19]:
test_loss = 0
s = 0.
for id_user in range(num_users):
  output = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = num_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(1.0062)
